# Run this cell to load all required libraries 

In [1]:
%matplotlib qt
import visa
from visa import VisaIOError
from csv import reader
import numpy

def parseCSVline(line):
    for lk in reader([line]):
        return lk
    
def avg(data):
    return sum(data)/len(data)

import time
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % (method.__name__, (te - ts) * 1000))
        return result
    return timed
    
class Instrument():
    '''This class adds automatic error checking to pyvisa write/query commands and also adds some common VXI/SCPI commands as class methods.'''
    def __init__(self, name, rm, address, full_test, visa_timeout_time=8000, visa_chunk_size=102400, no_check_commands=[], debug=True):
        self.name = name
        self.full_test = full_test
        self.debug = debug
        self.no_check_commands = no_check_commands #Commands which lock-up or reset the instrument, so checking for errors doesn't work
        self.resource = rm.open_resource(address)
        #Timeout needs to be long enough for some of the slow devices to respond
        self.resource.timeout = visa_timeout_time
        #Chunk just needs to be big enough for any/all data transfers
        self.resource.chunk_size = visa_chunk_size
        
    def query(self, cmd):
        '''Send a SCPI query and test/capture any instrument errors'''
        retval = None
        try:
            retval = self.resource.query(cmd)
        except Exception as e:
            #If the command fails/timeouts, check for errors
            raise Exception("Instrument Errors: "+repr(self.get_errors())+"\nLeading to "+str(e))
        #If it succeeds, still check for errors
        if cmd not in self.no_check_commands and self.debug:
            self.check_errors(cmd)
        return retval
    
    def write(self, cmd):
        '''Send a SCPI write and test/capture any instrument errors'''
        try:
            self.resource.write(cmd)
        except Exception as e:
            #If the command fails/timeouts, check for errors
            raise Exception("Instrument Errors: "+repr(self.get_errors())+"\nLeading to "+str(e))
        #If it succeeds, still check for errors
        if cmd not in self.no_check_commands and self.debug:
            self.check_errors(cmd)
    
    def get_errors(self):
        '''Get a list of errors of the device, if there's no error return an empty list'''
        errors=[]
        while True:
            try:
                errcode, errstring = parseCSVline(self.resource.query("SYSTEM:ERROR?"))
            except VisaIOError as e:
                #If the command fails/timeouts, check for errors
                raise Exception("VisaIOError on "+self.name+"\nLeading to "+str(e))
            errcode = int(errcode)
            if errcode == 0:
                #This isn't an error, all is fine so stop parsing here
                break
            errors.append([errcode, errstring])
        #Finally, we clear the instrument status if its errored
        if len(errors) != 0:
            self.resource.write("*CLS")
        return errors
    
    def check_errors(self, cmd=None):
        '''Checks if the instrument has any errors and, if so, raises an exception'''
        errors = self.get_errors()
        if len(errors) != 0:
            if cmd != None:
                raise Exception(self.name+": cmd:"+repr(cmd)+" gives errors "+repr(errors))
            else:
                raise Exception(self.name+": Errors "+repr(errors))
            
    ######## SCPI command helpers
    def reset(self):
        self.write("*RST")

class DMMX5XX(Instrument):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def fetchTrace(self, binary=True):
        self.write("*WAI")#Ensure trace is ready
        count = self.query(':TRACE:ACTUAL?').strip()
        
        if binary:
            self.write("FORM REAL")
            DMMdata = self.resource.query_binary_values('TRAC:DATA? 1, '+count+', "defbuffer1", READ, REL', is_big_endian=False, datatype=u'd')
            self.write("FORM ASCii")
        else:
            DMMdata = self.resource.query_ascii_values('TRAC:DATA? 1, '+count+', "defbuffer1", READ, REL')
        
        start = int(self.query(":TRACE:ACTUAL:START?"))
        end = int(self.query(":TRACE:ACTUAL:END?"))
        if end > start:
            sliceend = end
        else:
            sliceend = -1
        slice_data = DMMdata[2*start:2*sliceend]
        if sliceend == -1:
            slice_data = slice_data + DMMdata[:2*end]
        return [x-slice_data[1] for x in slice_data[1::2]], slice_data[0::2]
    
    def readData(self, t="VOLT:DC"):
        cmds = ["*RST",
        ':SENS:FUNC "'+t+'"',
        ":SENS:VOLT:RANGE:AUTO ON",
        ":SENS:VOLT:INP AUTO",
        ":SENS:VOLT:NPLC 1",
        ":SENS:VOLT:AZER ON"]
        for cmd in cmds:
            self.write(cmd)
        return float(self.query("READ?"))

    def readV(self):
        return self.readData("VOLT:DC")

    def readI(self):
        return self.readData("CURR:DC")
    
class Agilent66311B(Instrument):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def writeDisplay(self, val):
        PSU.write("DISPLAY:MODE TEXT")
        PSU.write('DISPLAY:TEXT "'+str(val)[:14]+'"')
    
    def resetDisplay(self):
        self.write("DISPLAY:MODE NORM")
    
    def readIArray(self, *args, **kwargs):
        return self.readDataArray("CURR", *args, **kwargs)
    
    def readVArray(self, *args, **kwargs):
        return self.readDataArray("VOLT", *args, **kwargs)
    
    def readDataArray(self, t="CURR", binary=True, count=2048, fetch_old=False):
        if not 0 < count <= 2049:
            raise RuntimeException("Bad data count of "+str(count))

        if fetch_old:
            get = "FETCH"
        else:
            get = "MEAS"
            self.write("SENS:SWE:POIN "+str(count))
        
        if binary:
            self.write("FORM REAL")
            val = self.resource.query_binary_values(get+":ARRAY:"+t+"?", is_big_endian=True)
            self.write("FORM ASCii")
            return val
        else:
            return self.resource.query_ascii_values(get+":ARRAY:"+t+"?")
        
    def readI(self, *args, **kwargs):
        return self.readData("CURR", *args, **kwargs)
    
    def readV(self, *args, **kwargs):
        return self.readData("VOLT", *args, **kwargs)
    
    def readData(self, t="CURR"):
        return list(map(float, [self.query("MEAS:"+t+"?"), self.query("FETC:"+t+":MAX?"), self.query("FETC:"+t+":MIN?")]))
    
rm = visa.ResourceManager()
DMMI = DMMX5XX('DMM6500', rm, 'TCPIP0::10.0.0.12::inst0::INSTR', full_test=False)
DMMV = DMMX5XX('DMM7510', rm,  'TCPIP0::10.0.0.8::inst0::INSTR', full_test=False)
PSU = Agilent66311B('66311B', rm, 'GPIB1::5::INSTR', full_test=False)

In [7]:
rm.list_resources()

('GPIB1::5::INSTR', 'TCPIP0::10.0.0.8::inst0::INSTR')

# Perform a validation of the PSU current (Switch DMM to front!)

In [2]:
PSU.write("*RST")
PSU.writeDisplay("Initialising...")
PSU.write("VOLT 1")
PSU.write("CURR 0.5")
PSU.write("OUTP ON")
PSU.write("*WAI")
#PSU.write("SENS:SWE:POIN 2048")
PSU.write("SENS:CURR:DET ACDC")
#PSU.write("SENS:CURR:RANG MAX")

import numpy as np

PSU.writeDisplay("Measuring")

VPSU=[]
IPSU=[]
IDMM=[]
for V in np.arange(0.1, 15.0, 0.5):
    VPSU.append(V)
    PSU.write("VOLT "+str(V))
    PSU.query("*OPC?")#Ensure completion of commands

    #Reading using the full averaging capabilities
    #dat = PSU.readI()
    #print(dat)
    #dat = dat[0]
    dat = PSU.readIArray(count=1)[0]
    
    IPSU.append(dat)
    IDMM.append(float(DMM.readV())/100.0)
    PSU.writeDisplay(("%0.4f" % IPSU[-1])+'->'+("%0.4f" % IDMM[-1]))
    #print(VPSU[-1], VDMM[-1])

PSU.writeDisplay("Plotting")
import pylab as plt
plt.plot([I*1000 for I in IPSU], [(x-y)*(1000) for x,y in zip(IPSU,IDMM)], 'xk')
plt.xlabel("PSU current ($mA$)")
plt.ylabel("$I_{PSU}-I_{DMM}$ (mA)")
plt.title("Validation of PSU current measurement")
plt.show()

PSU.write("VOLT 0")
PSU.write("OUTP OFF")
PSU.resetDisplay()

+1



NameError: name 'DMM' is not defined

# Run an acquisition of data (switch DMM to rear)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import math

#Our probe is this
#https://uk.rs-online.com/web/p/platinum-resistance-temperature-sensors/2364299/?relevancy-data=636F3D3126696E3D4931384E525353746F636B4E756D626572266C753D656E266D6D3D6D61746368616C6C26706D3D5E2828282872737C5253295B205D3F293F285C647B337D5B5C2D5C735D3F5C647B332C347D5B705061415D3F29297C283235285C647B387D7C5C647B317D5C2D5C647B377D2929292426706F3D3126736E3D592673723D2673743D52535F53544F434B5F4E554D4245522677633D4E4F4E45267573743D32333634323939267374613D3233363432393926&searchHistory=%7B%22enabled%22%3Atrue%7D
#1/5th DIN PT100 +-0.06C at 0C (in accordance with IEC 751)
#This means the alpha is 0.00385 
#Following the notes at http://educypedia.karadimov.info/library/c15_136.pdf
#We can use the Callendar-Van Dusen expression to calculate T
#
# Here's a definition via the ITS-90 versus the IPTS-68
#http://www.code10.info/index.php%3Foption%3Dcom_content%26view%3Darticle%26id%3D82:measuring-temperature-platinum-resistance-thermometers%26catid%3D60:temperature%26Itemid%3D83
def RTD_RtoT(R, R0=100):
    A = 3.9083E-3
    B = -5.775E-7
    #C = -4.183E-12
    Temp = (-R0 * A + math.sqrt(R0**2 * A**2 - 4 * R0 * B * (R0 - R)))  / (2 * R0 * B)
    return Temp

#Our wire lengths are 89.50mm and 60.50mm from pad end to pad end, and 0.015mm diameter
#Short and Long resistances per length should be within 2% (pg 463 NIST THW)
#Calculations for platinum with a resistivity of 10.6e-8 Ohm m, and diameter 0.015mm
#Gives 53.685438 Ohm for 8.95cm and 36.2901564 Ohm for 6.05cm        
wire_length_long = 0.0895 #m
wire_length_short = 0.0605 #m
# Measured in-place wire resistances are 56.3028 and 38.1656 with contact/lead resistance of 0.006, but much more variance in readings than that
def Long_HW_RtoT(R):
    Rpt =  1.9192360733223648 * R + 0.6772834578915337
    return RTD_RtoT(Rpt)

def Short_HW_RtoT(R):
    Rpt =  2.811359007746365 * R + 1.6831528533284796
    return RTD_RtoT(Rpt)


@timeit
def runTest(current, dt=15.6e-6, Rsense=100.0):
    Rlong = 60.0
    #dt above is the shortest value possible
    PSU.writeDisplay(str(current*1000)+"mA Init")
    #voltage=15
    #current = voltage / 200.0
    voltage = current * 200.0
    #Start the PSU
    PSU.write("VOLT 0.0")
    PSU.write("CURR 3") #Allow the max current 
    PSU.write("*WAI")
    PSU.write("OUTP ON")
    
    time.sleep(1)
    IVoffset = DMMI.readV()
    
    #Setup the voltage
    PSU.write("VOLT:TRIG "+str(voltage))
    PSU.write("INIT:SEQ1")

    #Setup the measurement
    PSU.write("TRIG:ACQ:SOUR BUS") #Trigger when the output is triggered

    #Below is waveform triggering
    #PSU.write("TRIG:ACQ:SOUR INT")
    #PSU.write("TRIG:ACQ:LEV:CURR 0.001")
    #PSU.write("TRIG:ACQ:HYST:CURR 0.001")#prevent spurious changes
    #PSU.write("TRIG:ACQ:SLOP:CURR POS")

    #We sense current from the PSU
    PSU.write('SENS:FUNC "CURR"') 
    PSU.write('SENS:SWE:POIN 4096') #Max 4096 samples allowed
    dt = 15.6e-6
    PSU.write('SENS:SWE:TINT '+str(dt)) #Set sample time
    #readback dt as the PSU rounds to the nearest 15.6e-6 s
    dt = float(PSU.query("SENSe:SWEep:TINTerval?"))
    PSU.write("SENS:SWE:OFFS:POIN 0") #from -4095 to 2e9 points offset from trigger

    #if current < 0.020:
    #    PSU.write("SENS:CURR:RANG MIN")
    #else:
    #    PSU.write("SENS:CURR:RANG MAX")
    PSU.write("SENS:CURR:RANG MAX")
        
    #PSU.write("SENS:CURR:DET DC") # For pure DC waveforms (not us! causes massive overshoot)
    PSU.write("SENS:CURR:DET ACDC") # for potential offsets of up to 2mA, but fast response

    PSU_srate = int(1.0/15.6e-6) #Sample rate per second (1/15.6e-6=64102.5641), max is 10^6
    oversample = 10
    DMMsamplerate = 100000 #PSU_srate*oversample
    DMMsamples = 400000
    cmds = ["*RST",
    ':SENS:DIG:FUNC "VOLT"',
    ':DIG:VOLT:RANGE 10',
    ":SENS:DIG:VOLT:SRATE "+str(DMMsamplerate),
    ":DIG:VOLT:ATR:EDGE:SLOP RIS",
    ":DIG:VOLT:ATR:MODE EDGE",
    ":DIG:VOLT:ATR:EDGE:LEV "+str(0.1*current*Rlong), # 1% of the predicted long wire V drop
    ':TRACe:POINts '+str(DMMsamples)+', "defbuffer1"',
    ':TRIGger:LOAD "LoopUntilEvent", ATRigger, 2, ENTer, 0, "defbuffer1"', #Loop until event keeping up to 2% of pre-trigger data
    ':DISPlay:SCReen GRAPh',
    '*WAI',
    ':INIT']
    
    for cmd in cmds:
        DMMV.write(cmd)

    cmds = ["*RST",
    ':SENS:DIG:FUNC "VOLT"',
    ':DIG:VOLT:RANGE 10',
    ":SENS:DIG:VOLT:SRATE "+str(DMMsamplerate), 
    ":DIG:VOLT:ATR:EDGE:SLOP RIS",
    ":DIG:VOLT:ATR:MODE EDGE",
    ":DIG:VOLT:ATR:EDGE:LEV "+str(0.1 * current * Rsense), #1% of the predicted sense resistor current drop
    ':TRACe:POINts '+str(DMMsamples)+', "defbuffer1"',
    ':TRIGger:LOAD "LoopUntilEvent", ATRigger, 2, ENTer, 0, "defbuffer1"', #Loop until event keeping up to 2% of pre-trigger data
    ':DISPlay:SCReen GRAPh',
    '*WAI',
    ':INIT']
    
    for cmd in cmds:
        DMMI.write(cmd)
    
    #Ready the trigger system for the run
    PSU.write("INIT:NAME ACQ")
    
    PSU.writeDisplay(str(current*1000)+"mA RUN")
    #Trigger the run
    PSU.write("*TRG")
    PSU.write("*WAI")
    
    PSU.writeDisplay(str(current*1000)+"mA SLEEP")
    time.sleep(DMMsamples / DMMsamplerate)
    
    #Disable the PSU
    PSU.write("VOLT 0")
    PSU.write("OUTP OFF")

    PSU.writeDisplay(str(current*1000)+"mA D/L V")
    t_V, V = DMMV.fetchTrace()
    PSU.writeDisplay(str(current*1000)+"mA D/L I")
    t_I, I = DMMI.fetchTrace()
    PSU.writeDisplay(str(current*1000)+"mA D/L I2")
    Ipsu = PSU.readIArray(fetch_old=True)
    t_Ipsu = [i * dt for i in range(len(Ipsu))]

    PSU.writeDisplay(str(current*1000)+"mA Proc")

    t0 = 0
    for ti,Vi in zip(t_V, V):
        if Vi > 0.1*current*Rlong:
            t0 = ti
            break
    t_V = [x-t0 for x in t_V]
    V = pd.Series(V, index=t_V)

    t0 = 0
    I = [(x - IVoffset) / Rsense for x in I] 
    for ti,Ii in zip(t_I, I):
        if Ii > 0.1*current:
            t0 = ti
            break
    t_I = [x-t0 for x in t_I]
    I = pd.Series(I, index=t_I)    
    
    t0=0
    for ti,Ii in zip(t_Ipsu, Ipsu):
        if Ii > 0.1*current:
            t0 = ti
            break
    #Here we do a time shift as it appears to take 900us for the supply to enable output after trigger
    t_Ipsu = [x-t0 for x in t_Ipsu]
    Ipsu = pd.Series(Ipsu, index=t_Ipsu)
    df = pd.DataFrame({'V':V, 'I':I, 'Ipsu':Ipsu})
    df.index.names = ['t']
    return df

plot = False
current=0.040

data = runTest(current=current)
if plot:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Raw measure data")
    data.V.plot(ax=ax, style="-", color="tab:red")
    ax2 = data.I.plot(secondary_y=True, ax=ax, style="-", color="tab:blue")
    data.Ipsu.plot(secondary_y=True, ax=ax, style=":", color="tab:blue")
    ax.set_ylabel("$V$ (V)", color="tab:red")
    ax2.set_ylabel("$I$ (A)", color="tab:blue")
    ax.tick_params(axis='y', labelcolor="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:blue")
    fig.legend()

PSU.writeDisplay("Plot interp data")
data = data.interpolate(method='zero')
if plot:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Interpolated measure data")
    data.V.plot(ax=ax, style="-", color="tab:red")
    ax2 = data.I.plot(secondary_y=True, ax=ax, style="-", color="tab:blue")
    data.Ipsu.plot(secondary_y=True, ax=ax, style="--", color="tab:blue")
    ax.set_ylabel("$V$ (V)", color="tab:red")
    ax2.set_ylabel("$I$ (A)", color="tab:blue")
    ax.tick_params(axis='y', labelcolor="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:blue")
    fig.legend()

#Trim off the zero time results
data = data[data.index > 0]
data['R'] = data.V / data.I
data['q'] = data.V * data.I / wire_length_long

if plot:
    PSU.writeDisplay("Plot R, Q")
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("Resistance")
    data['R'].plot(ax=ax, color="tab:red")
    ax2 = data['q'].plot(secondary_y=True, ax=ax, color="tab:blue")
    ax.set_ylabel("$R$ (Ohm)", color="tab:red")
    ax2.set_ylabel("$Q$ (W)", color="tab:blue")
    ax.set_xlabel("t (s)")
    ax.tick_params(axis='y', labelcolor="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:blue")

    PSU.writeDisplay("Plot T/DT/ln t")
    
data['T'] = data.R.apply(Long_HW_RtoT)
data.reset_index(level=0, inplace=True)
data['ln t'] = data['t'].transform(lambda t : numpy.log(t*1000))

if plot:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    data.plot('ln t', 'T', ax=ax)

data.to_excel('I'+str(current*1000)+'mA.xlsx')

PSU.writeDisplay("Complete!")

In [ ]:
#data.reset_index(level=0, inplace=True)
#data['ln t'] = data.t.apply(lambda t : math.log(t*1000.0))

#fig = plt.figure()
#ax = fig.add_subplot(111)
#data.plot('ln t', 'T', ax=ax)
#ax.set_ylabel("$R$ (Ohm)", color="tab:red")
#ax.set_xlabel("$\ln t$ (t in ms)")

#PSU.writeDisplay("Excel out")
#data.to_excel('raw.xlsx')
#PSU.writeDisplay("Done")